In [1]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.utils import class_weight
import datetime

import os

from tensorflow.keras import layers
from tensorflow.keras import Model

In [22]:
pre_trained_model = VGG16(include_top=True)

In [4]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [23]:
#Camera Model
vgg_conv_C = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_conv_C.layers:
    layer._name = layer._name + str('_C')
#Depth Model
vgg_conv_D = VGG16(weights='imagenet', include_top=False, input_shape= (224, 224, 3))
for layer in vgg_conv_D.layers:
    layer._name = layer._name + str('_D')
#Depth Model
vgg_conv_E = VGG16(weights='imagenet', include_top=False, input_shape= (224, 224, 3))
for layer in vgg_conv_E.layers:
    layer._name = layer._name + str('_E')

In [29]:
for layer in vgg_conv_D.layers[:-4]:
    layer.trainable = False 
for layer in vgg_conv_C.layers[:-4]:
    layer.trainable = False
for layer in vgg_conv_E.layers[:-4]:
    layer.trainable = False
mergedModel = layers.Concatenate()([vgg_conv_C.output, vgg_conv_D.output, vgg_conv_E.output])
mergedModel = Dense(units = 1024, activation='relu')(mergedModel)
mergedModel = BatchNormalization()(mergedModel)
mergedModel = Dropout(0.5)(mergedModel)
mergedModel = Dense(units = 1, activation = 'softmax')(mergedModel)
fused_model = Model([vgg_conv_C.input, vgg_conv_D.input, vgg_conv_E.input], mergedModel)   

In [8]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 512)


In [6]:
def get_RGB_ARRAY(ID_ARRAY):
    """ Outputs RGB values of picture 
        Shape of (N, 224, 224, 3) where N is size of input array. """

    N = np.size(ID_ARRAY, 0)
    temp_array = np.zeros((10,224,244,3))
    
    for ID in range(0,10):
        str_id = '{0:05}'.format(ID)
        img_path = 'food_porn/food/' + str_id + '.jpg'
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        temp_array[ID] = x

    return temp_array

In [12]:
def get_RGB(ID):
    """ Outputs RGB values of picture 
        Shape of (224, 224, 3) """

    
    str_id = '{0:05}'.format(ID)
    img_path = 'food_porn/food/' + str_id + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    return x

In [15]:
print(tripplet_train[0,1])

3450


In [14]:
data_x = get_RGB(tripplets_train[i,1] for i in range(0,59515))

==

get_RGB('059515')

TypeError: unsupported format string passed to generator.__format__

In [47]:
temp = get_RGB(tripplet_train[:,0])
print(temp.shape)

MemoryError: Unable to allocate 72.7 GiB for an array with shape (59515, 224, 244, 3) and data type float64

In [44]:
test = np.concatenate((get_RGB('00001'), get_RGB('00002')))

print(test.shape)

(2, 224, 224, 3)


In [ ]:
fused_model.fit((data_x,y,z), data_y, epochs=128, batch_size=1024, verbose=1, validation_split = 0.2)

In [17]:
# Flatten the output layer to 1 dimension
y = layers.Flatten()(last_output)

x = layers.concatenate([y, y, y])
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(pre_trained_model, x)

ValueError: Input tensors to a Functional must come from `tf.keras.Input`. Received: <tensorflow.python.keras.engine.functional.Functional object at 0x00000291740868E0> (missing previous layer metadata).

In [14]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      